<a href="https://colab.research.google.com/github/AlfredoMijares/Inventado/blob/main/Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from collections import Counter
import numpy as np

# Texto cifrado (sin espacios ni caracteres especiales)
ciphertext = "REIAUBLZXYQOKHMRNTEZHFLVIABHDMJMSJOGIRETPHBVVFTQHEXTARVIXSNQRIQPEJOLMDDEJORGVFECUTQVHIIKVMUKHVQPARPIVBIVGQIUFHIWZSTMBTHOSLXDNDFLFYIBPMRPOHCUOLJFEMSXIJBITHPSEQUXRZEEATPHDAFGLLQAXIQAKKRVFYTPHSVFGNLEQRVMTPWAXYQSCURETQONWTINMTMUMFFHEBKQVVPWMOXXYQSMDWMESAVGTMJEUJMQGKEWMPGWKZOBLYEXUNMWTEKFHMUQMJZOBKURXMTBKQFFFTWPAJKTEAHMFLFBIUQCVXLWZEEEP"  # Completar con todo el texto

# Frecuencia de letras en inglés
english_freqs = {
    'A': 0.082, 'B': 0.015, 'C': 0.028, 'D': 0.043, 'E': 0.13,
    'F': 0.022, 'G': 0.02, 'H': 0.061, 'I': 0.07, 'J': 0.002,
    'K': 0.008, 'L': 0.04, 'M': 0.024, 'N': 0.067, 'O': 0.075,
    'P': 0.019, 'Q': 0.001, 'R': 0.06, 'S': 0.063, 'T': 0.091,
    'U': 0.028, 'V': 0.01, 'W': 0.023, 'X': 0.001, 'Y': 0.02, 'Z': 0.001
}

# Función para calcular el Índice de Coincidencia
# Este código calcula el Índice de Coincidencia (IC) de un texto dado.
#El IC es una medida estadística utilizada en criptografía para analizar la
# distribución de las letras en un texto y detectar si es un texto cifrado con
# un cifrado monoalfabético o polialfabético.

def index_of_coincidence(text):
    N = len(text)
    freqs = Counter(text)
    IC = sum(f * (f - 1) for f in freqs.values()) / (N * (N - 1))
    return IC

# Función para estimar la longitud de la clave
# Este código implementa una función para estimar la longitud de la clave de un texto cifrado, utilizando el Índice de Coincidencia (IC).
# Se usa en el criptoanálisis de cifrados polialfabéticos como Vigenère, donde el IC puede ayudar a detectar el tamaño de la clave.

def estimate_key_length(text, max_key_length=10):
    ICs = []
    for key_length in range(1, max_key_length + 1):
        avg_IC = np.mean([index_of_coincidence(text[i::key_length]) for i in range(key_length)])
        ICs.append((key_length, avg_IC))
    return sorted(ICs, key=lambda x: -x[1])  # Ordenar de mayor a menor IC

# Determinar la clave usando análisis de frecuencia
# Este código implementa un ataque de análisis de frecuencia para determinar la clave de un texto cifrado con el cifrado Vigenère.
# Se basa en el cálculo de la prueba de chi-cuadrado para identificar el desplazamiento más probable de cada segmento del texto.
def find_vigenere_key(text, key_length):
    key = ""
    for i in range(key_length):
        segment = text[i::key_length]
        chi_squares = []
        for shift in range(26):
            shifted_text = ''.join(chr(((ord(c) - ord('A') - shift) % 26) + ord('A')) for c in segment)
            shifted_freqs = Counter(shifted_text)
            total_chars = len(segment)
            chi_square = sum(((shifted_freqs.get(chr(j + ord('A')), 0) / total_chars) - english_freqs[chr(j + ord('A'))]) ** 2 for j in range(26))
            chi_squares.append((shift, chi_square))
        best_shift = min(chi_squares, key=lambda x: x[1])[0]
        key += chr(best_shift + ord('A'))
    return key

# Descifrar texto cifrado con la clave obtenida
def decrypt_vigenere(text, key):
    key_length = len(key)
    decrypted_text = ""
    for i, char in enumerate(text):
        shift = ord(key[i % key_length]) - ord('A')
        decrypted_text += chr(((ord(char) - ord('A') - shift) % 26) + ord('A'))
    return decrypted_text

# Ejecución del ataque
estimated_keys = estimate_key_length(ciphertext)
best_key_length = estimated_keys[0][0]  # Tomar la mejor estimación
found_key = find_vigenere_key(ciphertext, best_key_length)
decrypted_text = decrypt_vigenere(ciphertext, found_key)

print(f"Longitud estimada de la clave: {best_key_length}")
print(f"Clave encontrada: {found_key}")
print(f"Texto descifrado: {decrypted_text}")
print(f"Texto organizado final: Far out in the ocean, where the water is as blue as the prettiest cornflower and as clear as crystal, it is very, very deep. So deep indeed that no cable could fathom it; many church steeples piled one upon another would not reach from the ground beneath to the surface of the water above. There dwell the Sea King and his subjects. We must not imagine that there is nothing at the bottom of the sea but bare yellow sand.")


Longitud estimada de la clave: 7
Clave encontrada: MERMAID
Texto descifrado: FAROUTINTHEOCEANWHERETHEWATERISASBLUEASTHEPRETTIESTCORNFLOWERANDASCLEARASCRYSTALITISVERYVERYDEEPSODEEPINDEEDTHATNOCABLECOULDFATHOMITMANYCHURCHSTEEPLESPILEDONEUPONANOTHERWOULDNOTREACHFROMTHEGROUNDBENEATHTOTHESURFACEOFTHEWATERABOVETHEREDWELLTHESEAKINGANDHISSUBJECTSWEMUSTNOTIMAGINETHATTHEREISNOTHINGATTHEBOTTOMOFTHESEABUTBAREYELLOWSAND
Texto organizado final: Far out in the ocean, where the water is as blue as the prettiest cornflower and as clear as crystal, it is very, very deep. So deep indeed that no cable could fathom it; many church steeples piled one upon another would not reach from the ground beneath to the surface of the water above. There dwell the Sea King and his subjects. We must not imagine that there is nothing at the bottom of the sea but bare yellow sand.
